# Dealing with Categorical Variables



## Learning Objectives



At the end of this notebook you should be able to... 
* find out which variables are categorical variables.
* transform a variable with label encoder and dummy variables.
* explain multicollinearity and avoid it when using dummy variables.


To figure all this out we need a sample dataset. Let's stick with the cars data we used for multiple linear regression. 

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# Import data
data = pd.read_csv('data/cars_multivariate.csv')
data.head()

In [ ]:
# Check data type for each column
data.info()

Except for `car name`, every other column seems to have numerical values. They could also be candidates to describe the dependent variable `mpg` (miles per gallon). On a first glance, it seems that we don't have any categorical variables.



## What are categorical variables?



Normally, we know the nature of the data and therefore we know which variables are categorical or numerical. But even if we don't know the data, we can find out if the variables could be categorical or numeric. Let's now take a closer look at the `origin` column.

In [ ]:
# Descriptive statistics for column origin
print(data['origin'].describe())

In [ ]:
# Number of unique values in column origin
print(data['origin'].nunique())

In [ ]:
# Unique values in column origin
print(data['origin'].unique())

Values range from 1 to 3, moreover, actually the only values that are in the dataset are 1, 2 and 3! it turns out that "origin" is a so-called **categorical** variable. It does not represent a continuous number but refers to a location - say 1 may stand for US, 2 for Europe, 3 for Asia (note: for this dataset the actual meaning is not disclosed).

Categorical variables, then, are exactly what they sound like: They represent categories, not numerical features. Note that, although not the case here, these features are often stored as text values representing different levels of observations.



## Identifying categorical variables



As categorical variables need to be treated in a particular manner, as you'll see later on, you need to make sure to identify which variables are categorical. 



In some cases, identifying will be easy (e.g. if they are stored as strings), in other cases they are numeric and the fact that they are categorical is not always immediately apparent.



Note that this may not be trivial. A first thing you can do is use the `.describe()` and `.info()` methods. `.describe()` will give you info on the data types (like strings, integers, etc), but even then continuous variables might have been imported as strings, so it's very important to really have a look at your data. 

In [ ]:
# Create plots to identify categorical features
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,3))

for xcol, ax in zip(['acceleration', 'displacement', 'horsepower', 'weight'], axes):
    data.plot(kind='scatter', x=xcol, y='mpg', ax=ax, alpha=0.4, color='b')

In [ ]:
# Create plots to identify categorical features
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(12,3))

for xcol, ax in zip([ 'cylinders', 'model_year', 'origin'], axes):
    data.plot(kind='scatter', x=xcol, y='mpg', ax=ax, alpha=0.4, color='b')

Note the structural difference between the top and bottom set of graphs. You can tell the structure looks very different: instead of getting a pretty homogeneous "cloud", categorical variables generate vertical lines for discrete values. Another plot type that may be useful to look at is the histogram.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
fig = plt.figure(figsize = (8,8))
ax = fig.gca()
data.hist(ax = ax);

And the number of unique values:

In [ ]:
data[['cylinders', 'model_year', 'origin']].nunique()

## Transforming categorical variables

When you want to use categorical variables in regression models, they need to be transformed. There are two approaches to this:
* Perform label encoding
* Create dummy variables / one-hot-encoding

### Label encoding

Let's illustrate label encoding and dummy creation with the following pandas Series with 3 categories: "USA", "EU" and "ASIA".

In [ ]:
origin = ['USA', 'EU', 'EU', 'ASIA','USA', 'EU', 'EU', 'ASIA', 'ASIA', 'USA']
origin_series = pd.Series(origin)

In [ ]:
origin_df = pd.DataFrame(origin,columns=['origin'])

Now you'll want to make sure Python recognizes there strings as categories. This can be done as follows:

In [ ]:
cat_origin = origin_series.astype('category')
cat_origin

Note how the `dtype` (i.e., data type) here is `category` and the three categories are detected.

Sometimes you'll want to represent your labels as numbers. This is called label encoding.

You'll perform label encoding in a way that numerical labels are always between 0 and (number_of_categories)-1. There are several ways to do this, one way is using `.cat.codes`

In [ ]:
cat_origin.cat.codes

Another way is to use scikit-learn's `OrdinalEncoder`:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ord_make = OrdinalEncoder()

origin_encoded = ord_make.fit_transform(origin_df[['origin']])

In [ ]:
origin_encoded

Note that while `.cat.codes` can only be used on variables that are transformed using `.astype(category)`, this is not a requirement to use `OrdinalEncoder`.

### Creating Dummy Variables

Another way to transform categorical variables is through using one-hot encoding or "dummy variables". The idea is to convert each category into a new column, and assign a 1 or 0 to the column. There are several libraries that support one-hot encoding, let's take a look at two:

In [ ]:
# Create dummy variables using pandas .get_dummies()
pd.get_dummies(cat_origin)

See how the label name has become the column name! Another method is through using the `LabelBinarizer` in scikit-learn. 

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
origin_dummies = lb.fit_transform(cat_origin)

# You need to convert this back to a dataframe
origin_dum_df = pd.DataFrame(origin_dummies,columns=lb.classes_)
origin_dum_df

The advantage of using dummies is that, whatever algorithm you'll be using, your numerical values cannot be misinterpreted as being continuous. Going forward, it's important to know that for linear regression (and most other algorithms in scikit-learn), **one-hot encoding is required** when adding categorical variables in a regression model!

## The Dummy Variable Trap



Due to the nature of how dummy variables are created, one variable can be predicted from all of the others. This is known as perfect **multicollinearity** and it can be a problem for regression. The basic idea behind perfect multicollinearity is that you can *perfectly* predict what one variable will be using some combination of the other variables. If this isn't super clear, go back to the one-hot encoded origin data above:

In [ ]:
# Create dummy variables using pandas .get_dummies()
trap_df = pd.get_dummies(cat_origin)
trap_df.head()

As a consequence of creating dummy variables for every origin, you can now predict any single origin dummy variable using the information from all of the others. OK, that might sound more like a tongue twister than an explanation so focus on the ASIA column for now. You can perfectly predict this column by adding the values in the EU and USA columns then subtracting the sum from 1 as shown below:

In [ ]:
# Predict ASIA column from EU and USA
predicted_asia = 1 - (trap_df['EU'] + trap_df['USA'])
predicted_asia.to_frame(name='Predicted_ASIA')

EU and USA can be predicted in a similar manner which you can work out on your own. 

You are probably wondering why this is a problem for regression. Recall that the coefficients derived from a regression model are used to make predictions. In a multiple linear regression, the coefficients represent the average change in the dependent variable for each 1 unit change in a predictor variable, assuming that all the other predictor variables are kept constant. This is no longer the case when predictor variables are related which, as you've just seen, happens automatically when you create dummy variables. This is what is known as the **Dummy Variable Trap**.

Fortunately, the dummy variable trap can be avoided by simply dropping one of the dummy variables. You can do this by subsetting the dataframe manually or, more conveniently, by passing ```drop_first=True``` to ```get_dummies()```: 

In [ ]:
pd.get_dummies(cat_origin, drop_first=True)

If you take a close look at the DataFrame above, you'll see that there is no longer enough information to predict any of the columns so the multicollinearity has been eliminated. 

You'll soon see that dropping the first variable affects the interpretation of regression coefficients. The dropped category becomes what is known as the **reference category**. The regression coefficients that result from fitting the remaining variables represent the change *relative* to the reference.

You'll also see that in certain contexts, multicollinearity and the dummy variable trap are less of an issue and can be ignored. It is therefore important to understand which models are sensitive to multicollinearity and which are not.

## Back to our cars data

Let's go ahead and change our `cylinders`, `model_year`, and `origin` columns to dummies and drop the first variable.

In [ ]:
# Create dummy variables for categorical columns
cyl_dummies = pd.get_dummies(data['cylinders'], prefix='cyl', drop_first=True)
yr_dummies = pd.get_dummies(data['model_year'], prefix='yr', drop_first=True)
orig_dummies = pd.get_dummies(data['origin'], prefix='orig', drop_first=True)

It could be that the order of the values in the `cylinders`, `model_year`, and `origin` columns plays a role. Feel free to check that using adjusted $R^2$, our results suggested the order does not matter. Next, let's remove the original columns from our data and add the dummy columns instead

In [ ]:
# Drop original columns
data = data.drop(['cylinders','model_year','origin'], axis=1)

In [ ]:
# Concatenate dummy variables to dataframe
data = pd.concat([data, cyl_dummies, yr_dummies, orig_dummies], axis=1)
data.head()

**Try it out yourself**

Can you find a combination of variables (also dummy variables) that beat your last model from Notebook 5?



## Summary



You now know
- that you need to look closely at your data to **find categorical variables** (EDA). You can do so by looking at the info and descriptive statistics as well as figure out how many unique values do each variable have. Also, scatter plots and histograms might help to find categorical variables.
- how to transform a variable with **label encoder** and **dummy variables**. For both ways you can use for example the [scikit-learns preprocessing tools](https://scikit-learn.org/stable/modules/preprocessing.html#encoding-categorical-features).
- what **multicollinearity** is and how to **avoid it when using dummy variables**. 
"Multicollinearity refers to a situation in which more than two explanatory variables in a multiple regression model are highly linearly related. We have perfect multicollinearity if, for example as in the equation above, the correlation between two independent variables is equal to 1 or −1." [Multicollinearity Wikipedia](https://en.wikipedia.org/wiki/Multicollinearity#:~:text=Multicollinearity%20refers%20to%20a%20situation,equal%20to%201%20or%20%E2%88%921.). To avoid collinearity when using dummy variables, you need to drop one dummy-column.